In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from IPython.display import clear_output

# Qiskit imports
from circuit_knitting.cutting import partition_problem
from qiskit.circuit import QuantumCircuit, ParameterVector
from qiskit.quantum_info import SparsePauliOp, PauliList
from qiskit.visualization import circuit_drawer
from qiskit_algorithms.optimizers.cobyla import COBYLA
from qiskit.algorithms.optimizers import ADAM, SPSA
from qiskit_algorithms.utils import algorithm_globals

In [ ]:
seed = 100

## Data Loading

In [ ]:
# Load data
data = pd.read_csv("diabetes_normalized.csv")
data = data.drop(["Unnamed: 0"], axis=1)
# data = data.drop(["BMI", "SkinThickness", "Pregnancies"], axis=1)
data.head(2)

In [ ]:
y = data["Outcome"]
x = data.drop(["Outcome"], axis=1)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [ ]:
x_train_A = x_train.iloc[:, :4]
x_train_B = x_train.iloc[:, 4:]
# x_train.iloc[:, 4:]

In [ ]:
x_test_A = x_test.iloc[:, :4]
x_test_B = x_test.iloc[:, 4:]

In [ ]:
# y_train

## Data Embedding

In [ ]:
# Data Embedding - Angle Encoding
def angle_encoding(feature_dims: int):
    embedding = QuantumCircuit(feature_dims)
    feature_param = ParameterVector("Theta", feature_dims)
    for qubit in range(feature_dims):
        embedding.ry(feature_param[qubit], qubit)
    return embedding, feature_param

In [ ]:
embedding, feature_params = angle_encoding(4)

## Tensor Network

In [ ]:
from ttn import TTN

In [ ]:
ttn = TTN(num_qubits=8).ttn_simple(complex_structure=False)
ttn.draw("mpl", style="iqp")

In [ ]:
# ttn.parameters

## Circuit Cutting

In [ ]:
observables = PauliList(["ZIIIIIII"])
partitioned_problem = partition_problem(circuit=ttn, partition_labels="AAAABBBB", observables=observables)
sub_circuits = partitioned_problem.subcircuits
sub_observables = partitioned_problem.subobservables
bases = partitioned_problem.bases

In [ ]:
sub_observables

In [ ]:
sub_circuits["A"].draw("mpl", style="iqp")

In [ ]:
sub_circuits["B"].draw("mpl", style="iqp")

In [ ]:
print(f"Sampling overhead: {np.prod([basis.overhead for basis in bases])}")

### Sub Experiments

In [ ]:
from circuit_knitting.cutting import generate_cutting_experiments

subexperiments, coefficients = generate_cutting_experiments(
    circuits=sub_circuits, observables=sub_observables, num_samples=np.inf
)

In [ ]:
# subexperiments
len(subexperiments["A"])

In [ ]:
subexperiments["A"][0].draw("mpl", style="iqp")

In [ ]:
subexperiments["B"][3].draw("mpl", style="iqp")

## Neural Network Training

In [ ]:
from qiskit_machine_learning.neural_networks import SamplerQNN, NeuralNetwork
from qiskit_aer.primitives import Sampler
from typing import Callable
from custom_sampler_qnn import CustomSampler

### Sampler

#### For subexperiments["A"]

In [ ]:
final_circuits = [embedding.compose(subex_circuit, inplace=False) for subex_circuit in subexperiments["A"]]
# final_circuits[0].draw("mpl")

In [ ]:
sampler_qcnn = CustomSampler(
    circuits=final_circuits, 
    input_params=feature_params.params,
    weight_params=sub_circuits["A"].parameters,
    input_gradients=False
)

In [ ]:
weights_A = algorithm_globals.random.random(7)
# forward_output, forward_sampler_result

forward_output = sampler_qcnn.forward(
    input_data=x_train_A,
    weights=weights_A,
)

In [ ]:
# forward output is a dictionary of 6 subex_circ items.
print(f"Output shape for {len(x_train_A)} samples: {forward_output[0].shape}")
print(len(forward_output[0]))
print(f"Output of the forward pass for first sample: \n{np.array([forward_output[i][0] for i in range(6)])}")

In [ ]:
input_grad, weights_grad = sampler_qcnn.backward(
    input_data=x_train_A,
    weights=weights_A
)

In [ ]:
print(f"Output shape for {len(x_train_A)} samples: {weights_grad[0].shape}")
print(f"Output of the backward pass for first sample for first subexperiment circuit: \n{np.array([weights_grad[i][0] for i in range(6)])}")

#### For subexperiments["B"]

In [ ]:
final_circuits2 = [embedding.compose(subex_circuit, inplace=False) for subex_circuit in subexperiments["B"]]

In [ ]:
sampler_qcnn2 = CustomSampler(
    circuits=final_circuits2, 
    input_params=feature_params.params,
    weight_params=sub_circuits["B"].parameters,
)

In [ ]:
weights_B = algorithm_globals.random.random(8)
forward_output2 = sampler_qcnn2._forward(
    input_data=x_train_B,
    weights=weights_B,
)

In [ ]:
print(f"Output shape for {len(x_train_B)} samples: {forward_output2[0].shape}")
print(len(forward_output2[0]))
print(f"Output of the forward pass for first sample: \n{np.array([forward_output2[i][0] for i in range(6)])}")

In [ ]:
input_grad2, weights_grad2 = sampler_qcnn2._backward(
    input_data=x_train_B,
    weights=weights_B
)

In [ ]:
print(f"Output shape for {len(x_train_B)} samples: {weights_grad[0].shape}")
print(f"Output of the backward pass for first sample for first subexperiment circuit: \n{np.array([weights_grad[i][0] for i in range(6)])}")

## Loss and Optimization

In [ ]:
from qiskit_machine_learning.utils.loss_functions import L2Loss
from qiskit_algorithms.optimizers import COBYLA, SPSA, GradientDescent
from custom_SPSA import CustomSPSA
from objective_func import CustomMultiClassObjectiveFunction
from optimization import create_objective, minimizer, print_optimizer_results

In [ ]:
def callback(nfev, params, fval, stepsize, accepted=None):
    """
    nfev: the number of function evals
    params: the current parameters
    fval: the current function value
    stepsize: size of the update step
    accepted: whether the step was accepted (not used for )
    """
    global objective_func_vals

    if (nfev % 3) == 0:
        objective_func_vals.append(fval)
        print(f"SPSA Epoch {len(objective_func_vals)}: {fval:.5f}")

In [ ]:
objective_func_vals = []
loss = L2Loss()
# optimizer = COBYLA(maxiter=10)
optimizer = CustomSPSA(maxiter=10, callback=callback)
# optimizer = GradientDescent(maxiter=2) # This doesn't work yet. The gradient shape doesn't match.

In [ ]:
# objective = create_objective(x_train_A, y_train, sampler_qcnn, loss)
initial_point = np.random.random((7,))
function = CustomMultiClassObjectiveFunction(x_train_A, y_train, sampler_qcnn, loss)

In [ ]:
function.objective0(initial_point)

In [ ]:
function.gradient0(initial_point)

In [ ]:
# initial_point = np.random.random((7,))
optimizer_result = minimizer(function, function.objective0, function.gradient0, initial_point, optimizer)

In [ ]:
print_optimizer_results(optimizer_result)

In [ ]:
plt.plot(objective_func_vals)
plt.xlabel("Number of epochs")
plt.title("Training loss")

## Reconstruct Expectation Values

In [ ]:
from circuit_knitting.cutting import reconstruct_expectation_values

In [ ]:
import copy

empty_dict = {}
for num_samples in range(537):
    empty_dict[num_samples] = []

dists_dict = copy.deepcopy(empty_dict)
metadata_dict = copy.deepcopy(empty_dict)

for num_samples in range(537):
    for k, v in forward_sampler_result.items():
        # print(num_samples, k, v.quasi_dists[num_samples])
        dists_dict[num_samples].append(v.quasi_dists[num_samples])
        metadata_dict[num_samples].append(v.metadata[num_samples])

A_dict = copy.deepcopy(empty_dict)
for key, item in new_dict.items():
    A_dict[key] = SamplerResult(quasi_dists=dists_dict[key], metadata=metadata_dict[key])

print(len(dists_dict), len(metadata_dict[0]))
print(len(A_dict), len(A_dict[0].quasi_dists), len(A_dict[0].metadata))

In [ ]:
# B_dict

dists_dict = copy.deepcopy(empty_dict)
metadata_dict = copy.deepcopy(empty_dict)

for num_samples in range(537):
    for k, v in forward_sampler_result2.items():
        # print(num_samples, k, v.quasi_dists[num_samples])
        dists_dict[num_samples].append(v.quasi_dists[num_samples])
        metadata_dict[num_samples].append(v.metadata[num_samples])

B_dict = copy.deepcopy(empty_dict)
for key, item in new_dict.items():
    B_dict[key] = SamplerResult(quasi_dists=dists_dict[key], metadata=metadata_dict[key])

print(len(dists_dict), len(metadata_dict[0]))
print(len(B_dict), len(B_dict[0].quasi_dists), len(B_dict[0].metadata))

In [ ]:
# combine dicts
combine_dict = {"A": None, "B": None}
reconstructed_expvals = []

zip_dict = zip(A_dict.items(), B_dict.items())
# print(zip_dict)
for (num_subex1, v1), (num_subex2, v2) in zip_dict:
    # print((num_subex1, v1), (num_subex2, v2))
    combine_dict["A"] = v1
    combine_dict["B"] = v2
    reconstructed_expvals.append(
        reconstruct_expectation_values(
            combine_dict,
            coefficients,
            sub_observables,
        )[0]
    )

# print(combine_dict)

In [ ]:
# reconstructed_expvals = reconstruct_expectation_values(
#     results,
#     coefficients,
#     subobservables,
# )